In [ ]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.15.0
Eager mode:  True
Hub version:  0.15.0
GPU is NOT AVAILABLE


In [ ]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="yelp_polarity_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"The Groovy P. and I ventured to his old stomping grounds for lunch today.  The '5 and Diner' on 16th St and Colter left me with little to ask for.  Before coming here I had a preconceived notion that 5 & Diners were dirty and nasty. Not the case at all.\\n\\nWe walk in and let the waitress know we want to sit outside (since it's so nice and they had misters).  We get two different servers bringing us stuff (talk about service) and I ask the one waitress for recommendations.  I didn't listen to her, of course, and ordered the Southwestern Burger w/ coleslaw and started with a nice stack of rings.\\n\\nThe Onion Rings were perfectly cooked.  They looked like they were prepackaged, but they were very crispy and I could actually bite through the onion without pulling the entire thing out (don't you hate that?!!!)\\n\\nThe Southwestern Burger was order Medium Rare and was cooked accordingly.  Soft, juicy, and pink with a nice crispy bro

In [ ]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([1, 0, 0, 0, 1, 1, 1, 1, 1, 1])>

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 1.2282207 , -0.0559625 , -0.21601607,  0.99687374,  0.26695687,
         0.05945293,  0.20330893,  0.04012216, -0.90225977,  0.573527  ,
         0.42579162,  0.29327655, -0.22316381,  0.55852324, -0.49511465,
        -0.33061767, -0.09551742,  0.3102484 , -0.00745646, -1.2499541 ,
         0.07989652, -0.6958521 ,  0.33565181,  0.31510636, -0.7297564 ,
        -0.0076625 , -1.2424903 ,  0.23516458,  0.3022343 , -0.56144404,
        -0.04623534,  0.57721865,  0.8767889 , -0.7155844 , -0.48821864,
         0.82578564,  0.05932135,  0.05675524,  0.49876395, -0.6845291 ,
         0.08247037,  0.09418593, -0.5611989 ,  0.335366  , -0.5268474 ,
         0.48554182, -0.59483457, -0.68769866, -0.24508585,  0.23467816],
       [ 0.36450046,  0.22738217, -0.09592111,  0.15292013, -0.19851975,
        -0.3464091 ,  0.33521134,  0.07005676, -0.42989457,  0.26057708,
        -0.04399252,  0.2049984 , -0.21570991, -0.14541769, -0.13797857,
 

In [ ]:
 model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=[], dtype=tf.string))  # Explicitly define input shape
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48191433 (183.84 MB)
Trainable params: 48191433 (183.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
657/657 [==============================] - 1041s 2s/step - loss: 0.2661 - accuracy: 0.8735 - val_loss: 0.1938 - val_accuracy: 0.9173
Epoch 2/10
657/657 [==============================] - 1069s 2s/step - loss: 0.1716 - accuracy: 0.9299 - val_loss: 0.1978 - val_accuracy: 0.9149
Epoch 3/10
657/657 [==============================] - 1064s 2s/step - loss: 0.1554 - accuracy: 0.9374 - val_loss: 0.2005 - val_accuracy: 0.9179
Epoch 4/10
657/657 [==============================] - 1066s 2s/step - loss: 0.1439 - accuracy: 0.9425 - val_loss: 0.2071 - val_accuracy: 0.9177
Epoch 5/10
657/657 [==============================] - 1066s 2s/step - loss: 0.1345 - accuracy: 0.9465 - val_loss: 0.2155 - val_accuracy: 0.9168
Epoch 6/10
657/657 [==============================] - 1127s 2s/step - loss: 0.1254 - accuracy: 0.9502 - val_loss: 0.2251 - val_accuracy: 0.9151
Epoch 7/10
657/657 [==============================] - 1102s 2s/step - loss: 0.1163 - accuracy: 0.9540 - val_loss: 0.2349 - val_accuracy:

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

75/75 - 17s - loss: 0.2820 - accuracy: 0.9125 - 17s/epoch - 224ms/step
loss: 0.282
accuracy: 0.912
